# 動的最適化

In [6]:
import numpy as np
import sys
from pulp import *
from functools import lru_cache
sys.setrecursionlimit(10000)

## 確定的動的最適化の例

### 再帰とメモ化

##### Fibonacci数
以下のように再帰的に定義される．  
$F_n = F_{n-1} + F_{n-2},$  
$F_1 = F_2 = 1$

In [2]:
def fibonacci(n):
    if n == 1 or n == 2:
        return 1
    else:
        return fibonacci(n-2) + fibonacci(n-1)

これは何回も同じ関数が呼ばれ，効率が悪い  
-> 引数としてmemoと名付けた辞書を渡し，一度計算した関数は記憶するようにする

In [8]:
def fibonacci_memo(n, memo={}):
    if n == 1 or n == 2:
        return 1
    elif n not in memo:
        memo[n] = fibonacci_memo(n-1) + fibonacci_memo(n-2)
    return memo[n]

In [20]:
fibonacci_memo(100)

354224848179261915075

デコレ―タ関数lru_cacheを用いるとさらに簡潔に書ける（lruはleast recently usedの略）

In [16]:
@lru_cache(maxsize=None)
def fibonacci_memo2(n):
    if n == 1 or n == 2:
        return 1
    else:
        return fibonacci_memo2(n-1) + fibonacci_memo2(n-2)

In [21]:
fibonacci_memo2(100)

354224848179261915075

### ナップサック問題

In [7]:
def create_toy_problem(size, seed):
    np.random.seed(seed)
    s = np.random.randint(2, 10, size=size)
    v = s + np.random.randint(-1, 3, size=size)
    return s, v

#### 整数ナップサック問題

Pulpから解く

In [10]:
s, v = create_toy_problem(size=100, seed=7)
b = 20
n = len(v)

prob = LpProblem(sense=LpMaximize)
x = [LpVariable(f'x{i}', lowBound=0, cat='Integer') for i in range(n)]
prob += lpSum(v[i] * x[i] for i in range(n))
prob += lpSum(s[i] * x[i] for i in range(n)) <= b

prob.solve()
print('Status: ', LpStatus[prob.status])
print(f'Optimal value = {value(prob.objective)}')

Status:  Optimal
Optimal value = 40.0


動的解法

In [70]:
def knapsack_problem(b):
    if b == 0:
        return 0
    if b < 0:
        return -99999
    max_value = 0
    for i, size in enumerate(s):
        max_value = max(max_value, knapsack_problem(b-size) + v[i])
    return max_value

In [72]:
s, v = create_toy_problem(size=20, seed=7)
knapsack_problem(b=10)

20

メモ化を用いた解法

In [57]:
def knapsack_problem_memo(b, memo={}):
    if b == 0:
        return 0
    if b < 0:
        return -99999
    if b in memo:
        return memo[b]
    else:
        max_value = 0
        for i in range(len(s)):
            max_value = max(max_value, knapsack_problem_memo(b-s[i]) + v[i])
        memo[b] = max_value
        return max_value

In [65]:
s, v = create_toy_problem(size=100, seed=7)
knapsack_problem_memo(300)

585

再帰を用いない前進型の動的最適化アルゴリズム

In [5]:
s, v = create_toy_problem(size=20, seed=7)
b = 10

# 各thetaに対して1つアイテムを足した時の値を見る
# 解が良くなるなら更新．結局すべてのthetaに対して最も良い解を求めている
f = {theta: 0 for theta in range(b+1)}
for theta in range(b-min(s)+1):
    for i, size in enumerate(s):
        if theta + size > b:
            continue
        if f[theta+size] < f[theta] + v[i]:
            f[theta+size] = f[theta] + v[i]

print('Optimal value: ', f[b])

Optimal value:  20


#### 0-1ナップサック問題

In [11]:
s, v = create_toy_problem(size=100, seed=7)
b = 20
n = len(v)

prob = LpProblem(sense=LpMaximize)
x = [LpVariable(f'x{i}', lowBound=0, cat='Binary') for i in range(n)]
prob += lpSum(v[i] * x[i] for i in range(n))
prob += lpSum(s[i] * x[i] for i in range(n)) <= b

prob.solve()
print('Status: ', LpStatus[prob.status])
print(f'Optimal value = {value(prob.objective)}')

Status:  Optimal
Optimal value = 38.0


In [16]:
def knapsack_problem_01(k, b, memo={}):
    if b < 0:
        return -99999
    if k == 0:
        if b >= s[0]:
            return v[0]
        else:
            return 0
    if (k, b) in memo:
        return memo[k, b]
    else:
        max_value = max(knapsack_problem_01(k-1, b), knapsack_problem_01(k-1, b-s[k]) + v[k])
        memo[k, b] = max_value
        return max_value

In [18]:
s, v = create_toy_problem(size=100, seed=7)
b = 20
k = len(v) - 1
knapsack_problem_01(k, b)

38

### 最大安定集合問題